In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/14 11:54:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/14 11:54:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Dataset - **Average Temperature of Mumbai** - [Link](https://www.kaggle.com/code/imdevskp/indian-cities-temperature/data?select=Mumbai.csv)

In [3]:
temp_cities = spark.read.csv('../Mumbai.csv',sep=',',header=True)

In [4]:
temp_cities.show()

+-----+---+----+-----------+
|MONTH|DAY|YEAR|TEMPERATURE|
+-----+---+----+-----------+
|    1|  1|1995|       71.8|
|    1|  2|1995|       72.0|
|    1|  3|1995|       70.3|
|    1|  4|1995|       69.7|
|    1|  5|1995|       71.3|
|    1|  6|1995|       73.9|
|    1|  7|1995|       72.9|
|    1|  8|1995|       70.2|
|    1|  9|1995|       68.4|
|    1| 10|1995|       68.0|
|    1| 11|1995|       69.9|
|    1| 12|1995|       69.8|
|    1| 13|1995|       71.0|
|    1| 14|1995|       73.9|
|    1| 15|1995|       73.4|
|    1| 16|1995|       70.4|
|    1| 17|1995|       74.9|
|    1| 18|1995|       74.5|
|    1| 19|1995|       73.0|
|    1| 20|1995|       71.9|
+-----+---+----+-----------+
only showing top 20 rows



**UDF to convert Fahreneit to Celcius**

In [5]:
from pyspark.sql.functions import *
@udf('double')
def convert_fahrenheit_to_celsius(fahrenheit):
    celsius = (fahrenheit - 32) * 5 / 9
    return celsius

In [7]:
temp_cities.dtypes

[('MONTH', 'string'),
 ('DAY', 'string'),
 ('YEAR', 'string'),
 ('TEMPERATURE', 'string')]

In [8]:
temp_cities_1=temp_cities.filter(col('TEMPERATURE')>0)\
                    .withColumn('MONTH',col('MONTH').cast('int')).withColumn('DAY',col('DAY').cast('int'))\
                    .withColumn('YEAR',col('YEAR').cast('int'))\
                    .withColumn('TEMPERATURE',col('TEMPERATURE').cast('double'))\
                    .withColumn('TEMPERATURE (In Celcius)',convert_fahrenheit_to_celsius(col('TEMPERATURE')))\
                    .withColumn('TEMPERATURE (In Celcius)',round(col("TEMPERATURE (In Celcius)"),2))\
                .withColumnRenamed('TEMPERATURE','TEMPERATURE (In Fahreneit)')
temp_cities_1.show()

+-----+---+----+--------------------------+------------------------+
|MONTH|DAY|YEAR|TEMPERATURE (In Fahreneit)|TEMPERATURE (In Celcius)|
+-----+---+----+--------------------------+------------------------+
|    1|  1|1995|                      71.8|                   22.11|
|    1|  2|1995|                      72.0|                   22.22|
|    1|  3|1995|                      70.3|                   21.28|
|    1|  4|1995|                      69.7|                   20.94|
|    1|  5|1995|                      71.3|                   21.83|
|    1|  6|1995|                      73.9|                   23.28|
|    1|  7|1995|                      72.9|                   22.72|
|    1|  8|1995|                      70.2|                   21.22|
|    1|  9|1995|                      68.4|                   20.22|
|    1| 10|1995|                      68.0|                    20.0|
|    1| 11|1995|                      69.9|                   21.06|
|    1| 12|1995|                  

**What Year was Temprature High and Low in the city?**

In [9]:
temp_cities_1.select('TEMPERATURE (In Celcius)').describe().show()

+-------+------------------------+
|summary|TEMPERATURE (In Celcius)|
+-------+------------------------+
|  count|                    8960|
|   mean|      27.833483258928542|
| stddev|      2.1517285494135843|
|    min|                   17.44|
|    max|                   33.67|
+-------+------------------------+



In [10]:
temp_cities_1.filter(col('TEMPERATURE (In Celcius)').isin(17.44,33.67)).show()

+-----+---+----+--------------------------+------------------------+
|MONTH|DAY|YEAR|TEMPERATURE (In Fahreneit)|TEMPERATURE (In Celcius)|
+-----+---+----+--------------------------+------------------------+
|    2|  7|2008|                      63.4|                   17.44|
|    6| 10|2014|                      92.6|                   33.67|
+-----+---+----+--------------------------+------------------------+



####  Find max and min temp temperature. Include records for only last day Of December every year

In [14]:
from pyspark.sql.window import *
w= Window().partitionBy("YEAR").orderBy(col('MONTH').desc(),desc(col('DAY')))

temp_cities_2=temp_cities_1.withColumn('rank',row_number().over(w))\
                    .filter(col('rank')==1)\
                    .drop('rank')
temp_cities_2.show(30)

+-----+---+----+--------------------------+------------------------+
|MONTH|DAY|YEAR|TEMPERATURE (In Fahreneit)|TEMPERATURE (In Celcius)|
+-----+---+----+--------------------------+------------------------+
|   12| 31|1995|                      75.9|                   24.39|
|   12| 31|1996|                      78.3|                   25.72|
|   12| 31|1997|                      78.4|                   25.78|
|   12| 29|1998|                      78.1|                   25.61|
|   12| 31|1999|                      79.6|                   26.44|
|   12| 31|2000|                      69.5|                   20.83|
|   12| 31|2001|                      79.2|                   26.22|
|   12| 31|2002|                      68.7|                   20.39|
|   12| 31|2003|                      70.5|                   21.39|
|   12| 31|2004|                      75.4|                   24.11|
|   12| 31|2005|                      76.7|                   24.83|
|   12| 31|2006|                  

In [15]:
max_temp=temp_cities_2.agg({'TEMPERATURE (In Celcius)':'max'}).collect()[0][0]
min_temp=temp_cities_2.agg({'TEMPERATURE (In Celcius)':'min'}).collect()[0][0]


temp_cities_2.filter(col('TEMPERATURE (In Celcius)').isin(max_temp,min_temp)).show()

+-----+---+----+--------------------------+------------------------+
|MONTH|DAY|YEAR|TEMPERATURE (In Fahreneit)|TEMPERATURE (In Celcius)|
+-----+---+----+--------------------------+------------------------+
|   12| 31|2002|                      68.7|                   20.39|
|    8| 11|2019|                      83.2|                   28.44|
+-----+---+----+--------------------------+------------------------+

